In [1]:
import lxml.html as html
import io
import nltk
import string
import math

texts_auto = []
texts_research = []


base_auto = 'http://www.autonews.ru'
for page in range(1,10):
    main_page = html.parse('http://www.autonews.ru/news/list/' + str(page) + '/')
    urls = main_page.getroot().xpath('//ul[@class="news-list__list"]//a[@href]/@href')

    for url in urls:
        page = html.parse(base_auto + url)
        # Getting article header
        text = page.getroot().xpath('//*[contains(@class, "article__")]/text()')
        text = [''.join(t).encode('utf-8', 'ignore').decode('utf-8') for t in text]
        page_text = ''.join(text)
        page_text = page_text.replace('\n', ' ').replace('\r', '')
        page_text = " ".join(page_text.split())
        texts_auto.append(page_text)
print len(texts_auto)

for page in range(2,10):
    main_page = html.parse('http://www.marketing.rbc.ru/news_research/p' + str(page) + '/')
    urls = main_page.getroot().xpath('//div[@class="list"]//a[@href]/@href')


    for url in urls:
        page = html.parse(url)
        # Getting article header
        text = page.getroot().xpath('//*[contains(@class, "newsBlock")]//p/text()')
        text = [''.join(t).encode('utf-8', 'ignore').decode('utf-8') for t in text]
        page_text = ''.join(text)
        page_text = page_text.replace('\n', ' ').replace('\r', '')
        page_text = " ".join(page_text.split())
        texts_research.append(page_text)
print len(texts_research)

234
240


In [2]:
def load_ngrams(file):
    input_f = io.open(file, "r", encoding='utf8')
    n_dict = {}
    for line in input_f:
        line = line.replace('\n', '').split('\t')
        n_dict[tuple(line[0:])] = int(line[0])
    input_f.close()
    return n_dict

def tokenize(corpus):
    news_list = corpus
    words = []
    for text in news_list:
        text_str = ''.join(text)
        line = nltk.word_tokenize(text_str)
        line = [word.encode("utf-8").decode("utf-8").lower() for word in line if word not in string.punctuation
                        and word not in [u'\u2012', u'\u2013', u'\u2014', u'\u2015', u'``', u"''"]]
        words += line
    return words

def find_perplexity(category, n3_gram, n2_gram):
    sum_probability = 0
    unknown_words = 0
    min_probability = 1
    sum_log = 0

    current_probability= {}

    for i in range(2, len(category)):
        try:
            current_probability[i] = (n3_gram[(category[i-2], category[i-1], category[i])] + 0.0)/n2_gram[(category[i-2],category[i-1])]
            sum_probability += current_probability
            if current_probability[i] < min_probability:
                min_probability = current_probability[i]
        except KeyError:
            unknown_words +=1
    print (unknown_words, len(category), sum_probability)
    known_words = len(category) - unknown_words
    min_probability /= 10.0
    probability_to_substract = (unknown_words * min_probability + 0.0) / known_words
    for i in range(2, len(category)):
        try:
            sum_log += math.log(current_probability[i] - probability_to_substract, 2)
        except KeyError:
            sum_log += math.log(min_probability, 2)

    return sum_log

In [4]:
n_2_grams = load_ngrams("2grams-3.txt")

In [5]:
n_3_grams = load_ngrams("3grams-3.txt")

In [6]:
tokenized_auto = tokenize(texts_auto)
tokenized_research = tokenize(texts_research)

In [7]:
perplexity_auto = find_perplexity(tokenized_auto, n_2_grams, n_3_grams)
perplexity = 2**(-perplexity_auto / len(tokenized_auto))
print("Perplexity for AUTO: {}".format(perplexity))
perplexity_research = find_perplexity(tokenized_research, n_2_grams, n_3_grams)
perplexity = 2**(-perplexity_research / len(tokenized_research))
print("Perplexity for RESEARCH: {}".format(perplexity))

(54643, 54645, 0)
Perplexity for AUTO: 9.99915729233
(45382, 45384, 0)
Perplexity for RESEARCH: 9.99898533921


#------------------------------------------------------------------------------------------
Предсказательная способность вероятностной модели корпуса для темы RESEARCH незначительно превосходит подобный показатель для темы AUTO.